## Prosta implementacja k-nearest-neighbors

In [1]:
import numpy as np
from collections import Counter

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_one(x) for x in X])

    def _predict_one(self, x):
        # oblicz odległość do wszystkich punktów w zbiorze treningowym
        distances = [
            self._euclidean_distance(x, x_train)
            for x_train in self.X_train
        ]

        # znajdź indeksy k najbliższych sąsiadów
        k_indices = np.argsort(distances)[:self.k]

        # znajdź etykiety k najbliższych sąsiadów
        k_labels = self.y_train[k_indices]

        # głosowanie
        most_common = Counter(k_labels).most_common(1)
        return most_common[0][0]


### Test na zbiorze MNIST

In [2]:
from torchvision import datasets, transforms

transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# przygotowanie danych
def dataset_to_numpy(dataset):
    X = []
    y = []

    for i, (img, label) in enumerate(dataset):
        X.append(img.view(-1).numpy())  # 28x28 -> 784
        y.append(label)

    return np.array(X), np.array(y)

X_train, y_train = dataset_to_numpy(train_dataset)
X_test, y_test = dataset_to_numpy(test_dataset)

print("MNIST załadowany!")

MNIST załadowany!


In [3]:
# UWAGA: Powyższa implementacja KNN jest bardzo wolna - u mnie ta komórka wykonywała się już 20 minut i się poddałem...

model = KNN()
model.fit(X_train, y_train)
preds = model.predict(X_test)

correct = np.sum(preds == y_test)
total = len(y_test)

print(f"Dokładność na teście: {100 * correct / total:.2f}%")

KeyboardInterrupt: 

In [5]:
# Szybsza implementacja z biblioteki sklearn - komórka wykonywała się kilka sekund

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
preds = model.predict(X_test)

correct = np.sum(preds == y_test)
total = len(y_test)

print(f"Dokładność na teście: {100 * correct / total:.2f}%")

Dokładność na teście: 97.05%


### Test Random Forest na zbiorze MNIST

In [7]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

model.fit(X_train, y_train)
preds = model.predict(X_test)

correct = np.sum(preds == y_test)
total = len(y_test)

print(f"Dokładność na teście: {100 * correct / total:.2f}%")

Dokładność na teście: 97.04%
